questa prima parte va ad importare tutto quello che mi servirà dopo e fa partire Chrome

In [1]:
from selenium import webdriver                         # import the webdriver API 
from selenium.webdriver.common.keys import Keys        # import the keys of the keyboard such as RETURNS, F1, ALT, ...
import re                                              # import regular expression

from selenium.common.exceptions import NoSuchElementException#importo l'eccezione, in modo che la possa riconoscere
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC


from selenium.webdriver.chrome.options import Options
chromeOptions = Options()  
chromeOptions.add_argument("--headless") 
driver = webdriver.Chrome(options=chromeOptions) # lancio il driver in modalità Headless

#driver = webdriver.Chrome()                            # run the browser
                                           

Adesso vado a recuperare i links salvati nel file "dataLinks.json" e li stampo a schermo

In [2]:
import json

links = []    # dichiaro l'array links

with open('dataLinks.json') as json_file:
    dataLinks = json.load(json_file)
    print(dataLinks)
    
    print("***************************")

    for link in dataLinks:
        print(link["link"])
        
        links.append(link["link"])    # metto i links recuperati da "dataLinks", in "links"  


[{'link': 'http://sira.arpat.toscana.it/apex2/f?p=QUALACQUA_MAS'}, {'link': 'http://sira.arpat.toscana.it/apex2/f?p=QUALACQUA_MAT'}, {'link': 'http://sira.arpat.toscana.it/apex2/f?p=QUALACQUA_MAR'}, {'link': 'http://sira.arpat.toscana.it/apex2/f?p=QUALACQUA_POT'}, {'link': 'http://sira.arpat.toscana.it/apex2/f?p=QUALACQUA_VTP'}, {'link': 'http://sira.arpat.toscana.it/apex2/f?p=QUALACQUA_VTM'}, {'link': 'http://sira.arpat.toscana.it/apex2/f?p=QUALACQUA_BAL'}, {'link': 'http://sira.arpat.toscana.it/apex2/f?p=QUALACQUA_NIT'}, {'link': 'http://sira.arpat.toscana.it/apex2/f?p=QUALACQUA_FIT'}]
***************************
http://sira.arpat.toscana.it/apex2/f?p=QUALACQUA_MAS
http://sira.arpat.toscana.it/apex2/f?p=QUALACQUA_MAT
http://sira.arpat.toscana.it/apex2/f?p=QUALACQUA_MAR
http://sira.arpat.toscana.it/apex2/f?p=QUALACQUA_POT
http://sira.arpat.toscana.it/apex2/f?p=QUALACQUA_VTP
http://sira.arpat.toscana.it/apex2/f?p=QUALACQUA_VTM
http://sira.arpat.toscana.it/apex2/f?p=QUALACQUA_BAL
http:/

questa parte seleziona i link di mio interesse ed apre tutti i tab contenenti quelle pagine

In [3]:
    # parameters and output data
#    url         = links[0]  # link alla pagina
    id_n_pulsante_m = "apexir_ACTIONSMENUROOT"  # id del pulsante menu a tendina 
    id_n_menu_tenina = "apexir_ACTIONSMENU"     # id menu a tendina 
        
#    driver.get(url)  # get the web page
    select = re.compile("(\w*MAS$)|(\w*MAT$)|(\w*POT$)")    #espressione regolare per selezionare i link di mio interesse
    i = 0    #la prima tab
    
    for link in links:
        if (select.search(links[i])):# usa select come valore di controllo, vede se corrisponde ad una delle opzioni
            driver.execute_script("window.open('about:blank', 'tab"+str(i)+"');")
            driver.switch_to.window("tab"+str(i))
            driver.get(links[i])
        
        i+=1
# ATT mettere del codice che ripete un ciclo su tutti i tab, e per ognuno fa partire la procedura di download,
# una volta fatta partire, passa a quello dopo e fa lo stesso

# DUBBIO, penso che partito uno, aspetta inizio download per passare a quello dopo. 
# Questo renderebbe inutile questa cosa. 
# Se fosse così a quel punto, potrei usare questo ciclo for anche per scaricare 
  
    
#    id_t_csv = "apexir_CONTROL_PANEL_DROP"   # individua la tabella csv html 
    
#    t_csv = driver.find_element_by_id(id_t_csv)  # individua il menu a tendina
#    t_csv.find_element_by_css_selector("a[title='CSV']").click()  # fa click su scarica  
    


la parte sotto trova i filtri e deve toglierli

ATT --> anche questa parte qui viene influenzata dall'aspettare che la pagina carichi, quindi applicare lo stesso metodo poi trovato anche qui

In [4]:
# inizio parte che individua e toglie i filtri *****************************************************************
temp = driver.find_elements_by_css_selector("img[title='Rimuovi filtro']")
len(temp)   

1

In [5]:
    tmp = temp
    try:
        while (len(tmp) != 0):#tmp != []
            driver.find_element_by_css_selector("img[title='Rimuovi filtro']").click()
            tmp = driver.find_elements_by_css_selector("img[title='Rimuovi filtro']")
    except:
        try:
            while (len(tmp) != 0):#tmp != []
                driver.find_element_by_css_selector("img[title='Rimuovi filtro']").click()
                tmp = driver.find_elements_by_css_selector("img[title='Rimuovi filtro']")            
        except:
            print("An exception occurred")

# fine parte che gestisce i filtri *******************************************************************************

An exception occurred


Questo mi fa muovere tra i vari tab (magari lo faccio ciclare per girarmeli tutti)

In [6]:
driver.switch_to.window(driver.window_handles[3]) #mi fa muovere tra i tab

Questo serve a scaricare la pagina usando wget

si rende necessario in quanto le coordinate geografiche non hanno un link scaricabile, ma un link alla pagina con i dati

In [7]:
def coord():
        #****************************** nella pagina dove sono, cerca il link con quel nome
        #questo è inserito nel ctrl_t in modo che quando fa i vari controlli per scaricare, controlla anche questo
        #inoltre per semplicità va inserito come ultimo comando, cioé
        #fa tutto il resto, poi questo, e poi cambia tab 
    import wget 
    
    driver.find_element_by_link_text('GEOJSON').click()
    #url = driver.page_source
    url = driver.current_url
    print(url)
    wget.download(url)#,out="") # download
        
        # la versione sotto funziona ma scarica tutto, compresi i tag html che mi danno fastidio
        #with open('page.geojson', 'w') as f:
        #    f.write(driver.page_source)
        
        #******************************

In [8]:
#dirgli in qualche maniera di controllare se nella pagina è presente un tag <a> con valore testo "GEOJSON"
#se c'è di scaricarlo 

def ctrl_t():
#from selenium.common.exceptions import NoSuchElementException
    try:

        elem = driver.find_element_by_id(id_n_pulsante_m)
        # inizio parte che trova ed apre il menu a tendina 

        driver.find_element_by_id(id_n_pulsante_m).click()     # seleziono il pulsante menu a tendina
    
        tendina = driver.find_element_by_id(id_n_menu_tenina)  # individua il menu a tendina
        tendina.find_element_by_css_selector("a[title='Scarica']").click()  # fa click su scarica
        # a questo punto il dawnload deve ancora partire, lo fa la funzione "get_CSV()" 

        # fine parte che apre il menu a tendina
        #print("vado ad eseguire coord")
        #coord()
        print("sono arrivato in fondo al primo caso")
    
    except NoSuchElementException:   # eccezione in quanto è una sola la pagina di questo tipo, diversa dalle altre
        #controlla e scarica i due file da link
        links_div = driver.find_elements_by_class_name("t20CVS")    # individua i link e li mette in links_div
        for i in links_div:
            print(i)
            i.find_element_by_tag_name('a').click()    # fa click sui link per scaricarli
        coord()
        print("fine della eccezione")

In [9]:
#va messo in un for che mi cicla tutti i tab
ctrl_t()

sono arrivato in fondo al primo caso


 Questa parte qui sotto va bene cosi serve a scaricare il csv
 

Per poter eseguire questa parte qui, è sufficiente commentare coord e continuare ad eseguire il codice per vedere se get_CSV() fa quello che deve 

Dopo andrà comunque resa eseguibile normalmente insieme a coord() in ctrl_t() 

è questo il punto problematico visto che il server della regione impiega troppo tempo a rispondere qui

In [10]:
def get_CSV():
   
    id_csv = "apexir_dl_CSV"                    # individua l'id del csv 
    WebDriverWait(driver, 5)
    driver.find_element_by_id(id_csv).click()   # scarica il csv
    
    return 1


anche qui forse ci vorrebbe un piccolo tempo di attesa 

verificare se eseguendolo headless non è necessario

In [12]:
get_CSV()
#driver.implicitly_wait(600)

KeyboardInterrupt: 

webdriver.get('about://blank')
my_script = 'var a = document.createElement("a");' \
            'var linkText = document.createTextNode("%s");' \
            'a.appendChild(linkText);' \
            'a.title = "%s";' \
            'a.href = "%s";' \
            'document.body.appendChild(a);' % \
            (url_to_use, url_to_use, url_to_use)
webdriver.execute_script(my_script)
webdriver.set_page_load_timeout(20)
webdriver.click_element_by_text('css=a', url_to_use)
if page.loaded() is False:
    webdriver.click_element_by_text('css=a', url_to_use)
if page.loaded() is False:
    webdriver.click_element_by_text('css=a', url_to_use)
if page.loaded() is False:
    webdriver.click_element_by_text('css=a', url_to_use)

In [ ]:
driver.quit()